# 字典

## 广义映射类型

In [1]:
from collections import abc
d = {}
isinstance(d, abc.Mapping)

True

## 字典构造

In [2]:
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

## 字典推导

In [3]:
CODES = [
    (86, 'China'),
    (91, 'India'),
    (1, 'USA')
]
{country: code for code, country in CODES}

{'China': 86, 'India': 91, 'USA': 1}

## 弹性键查询

### 1, defaultdict

在实例化一个 defaultdict 的时候，需要给构造方法提供一个可调用对象，这个可调用对象会在 `__getitem__` 找不到键的时候被调用，以便 `__getitem__` 返回默认值。

In [4]:
import collections
d = collections.defaultdict(list)
d['a'].append('b')
d['c'].append('d')
d

defaultdict(list, {'a': ['b'], 'c': ['d']})

### 2, 特殊方法 \_\_missing\_\_

映射类型在处理找不到的键的时候，都会涉及 `__missing__` 方法。
虽然基类 dict 没有定义这个方法，但是如果一个类继承了 dict ，然后提供了 `__missing__` 方法，那么当 `__getitem__` 遇到找不到键的时候，Python 会自动调用它，而不是抛出 KeyError 异常。

**\_\_missing\_\_ 方法只会被 `__getitem__` 调用（即使用表达式 `d[k]`）**

## collections.OrderedDict

这个类型在添加键的时候会保持顺序，因此键的迭代次序总是一致的。

OrderedDict 的 popitem 方法默认删除并返回字典里最后一个元素，但是如果调用 `popitem(last=False)` ，则删除并返回第一个被添加进去的元素。

## collections.ChainMap

该类型可以容纳多个不同的映射对象，在进行键查找操作时，会**逐个**查找这些映射对象，直到键被找到为止。

这个功能在给有嵌套作用域的语言做解释器的时候很有用，可以用一个映射对象来代表一个作用域上下文。

In [5]:
import builtins
from collections import ChainMap
pylookup = ChainMap(locals(), globals(), vars(builtins))

## collections.Counter

这个映射类型会给键准备一个整数计数器，每次更新一个键的时候会增加这个计数器。

Counter 实现了 + 和 - 运算符来合并记录。`most_common([n])` 方法会返回最常见的 n 个键和它们的计数。

In [6]:
from collections import Counter
ct = Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'c': 1, 'd': 1, 'r': 2})

In [7]:
ct.update('aaaaazzz')
ct

Counter({'a': 10, 'b': 2, 'c': 1, 'd': 1, 'r': 2, 'z': 3})

In [8]:
ct.most_common(2)

[('a', 10), ('z', 3)]

## collections.UserDict

这个类其实是把标准 dict 用纯 Python 又实现了一遍，主要的用途是让用户继承写子类的。

更倾向于从 UserDict 而不是从 dict 继承的主要原因是后者有时会在某些方法的实现上走一些捷径，导致不得不在子类中重写这些方法（比如 dict 的子类实现的 `__getitem__` 方法不会被 `get()` 方法所调用），但是 UserDict 就不会有这个问题。

In [11]:
class MyDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value*2)

d = MyDict(one=1)
d

{'one': 1}

In [13]:
d['two'] = 2
d

{'one': 1, 'two': 4}

继承自 dict 的 `__init__` 方法忽略了子类的 `__setitem__` 方法。

In [14]:
d.update(three=3)
d

{'one': 1, 'three': 3, 'two': 4}

继承自 dict 的 `update` 方法也忽略了子类的 `__setitem__` 方法。

另外一个值得注意的地方是，UserDict 并不是 dict 的子类。UserDict 有一个 data 属性，是 dict 的实例，这个属性实际上是 UserDict 最终存储数据的地方。

## 不可变映射类型

types.MappingProxyType 会返回一个只读的映射视图。虽然是只读视图，但是它是动态的，如果对原映射做出改动，通过这个视图可以观察到，但是无法通过这个视图对原映射做出修改。

In [15]:
from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [16]:
d_proxy[1]

'A'

In [20]:
try:
    d_proxy[2] = 'x'
except Exception as e:
    print(e)

'mappingproxy' object does not support item assignment


In [21]:
d[2] = 'B'
d_proxy

mappingproxy({1: 'A', 2: 'B'})